Se utilizará como conjunto de datos un archivo de texto donde se registran los accesos (log) al servidor web Apache del repositorio SEDICI. 

In [1]:
from apachelogs import LogParser
parser = LogParser("%h %l %u %t \"%r\" %>s %b \"%{Referer}i\" \"%{User-Agent}i\"")

A continuación se muestra un ejemplo de como se registra un acceso y los datos que se pueden obtener del mismo:

In [2]:
example = '54.36.148.250 - - [08/Nov/2021:00:00:14 -0300] "GET /bitstream/handle/10915/37522/Tesis%20Ignacio%20Perez%20de%20Berti.pdf.jpg?isAllowed=y&sequence=3 HTTP/1.1" 200 4737 "-" "Mozilla/5.0 (compatible; AhrefsBot/7.0; +http://ahrefs.com/robot/)"'
entry = parser.parse(example)
print("IP del cliente: " + entry.directives["%h"])
print("Identidad del usuario determinado por identd: " + str(entry.directives["%l"]))
print("Nombre de usuario determinado por autenticación HTTP: " + str(entry.directives["%u"]))
print("Fecha en la que el servidor termina de procesar el request: " + str(entry.directives["%t"]))
print("Request del cliente: " + str(entry.directives["%r"]))
print("Código de respuesta que devuelve el servidor al cliente: " + str(entry.directives["%>s"]))
print('Referer (URI desde la que se realiza la solicitud):' + str(entry.headers_in["Referer"]))
print('Navegador que hace la petición:'+ str(entry.headers_in["User-Agent"])) 
print("tamaño de la respuesta al cliente (en bytes):" + str(entry.directives["%b"]))

IP del cliente: 54.36.148.250
Identidad del usuario determinado por identd: None
Nombre de usuario determinado por autenticación HTTP: None
Fecha en la que el servidor termina de procesar el request: 2021-11-08 00:00:14-03:00
Request del cliente: GET /bitstream/handle/10915/37522/Tesis%20Ignacio%20Perez%20de%20Berti.pdf.jpg?isAllowed=y&sequence=3 HTTP/1.1
Código de respuesta que devuelve el servidor al cliente: 200
Referer (URI desde la que se realiza la solicitud):None
Navegador que hace la petición:Mozilla/5.0 (compatible; AhrefsBot/7.0; +http://ahrefs.com/robot/)
tamaño de la respuesta al cliente (en bytes):4737


Vamos a presindir de la identidad del usuario (definida por el protocolo identd) y del nombre de usuario autenticado ya que todos los datos explorados resultaron ser nulos. Para eso se creará un Dataframe, de la librería [Pandas](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html), que irá almacenando de cada uno de los eventos de acceso al servidor los siguientes datos:

* IP del cliente
* fecha en la que el servidor termina de procesar el request
* request del cliente
* código que define el estado de la respuesta que el servidor devuelve al cliente
* tamaño de la respuesta al cliente (en bytes)

In [4]:
import pandas as pd, sys
import os
print()

path = os.path.realpath(os.path.join(os.path.dirname(os.path.abspath("__file__")), 'data/access-dspace80.log'))


file = open(path)
lines = file.readlines()

df_log = pd.DataFrame({'ip': [], 'time': [], 'request': [], 'status': [], 'size': [], 'Referer': [],'User_agent': [] })
lines = open(path).readlines()


Itero por cada línea del log y voy construyendo un dataframe llamado df_log

In [ ]:
for index, entry in enumerate(lines):
    try:
        data = parser.parse(entry);
    except:
        sys.stderr.write("Unable to parse %s" % entry)
    ip = data.directives["%h"]
    time = data.directives["%t"]
    request = data.directives["%r"]
    status = data.directives["%>s"]
    size = data.directives["%b"]
    referer = data.headers_in["Referer"]
    user_agent = data.headers_in["User-Agent"]
    aux = pd.DataFrame([[ip, time, request, status, size, referer, user_agent]], columns=['ip', 'time', 'request', 'status', 'size', 'referer','user_agent'])
    df_log = pd.concat([df_log, aux])

Almaceno df_log en un archivo parquet para ser procesado más adelante

df_log = df_log.convert_dtypes()
df_log.to_parquet('data/df_log.parquet.gzip', compression='gzip', index=False)

El dataframe quedo de la siguiente manera

In [9]:
df_log

,ip,time,request,status,size,Referer,User_agent
0,3.224.220.101,2022-03-31 00:00:13-03:00,GET /handle/10915/60216/browse?authority=http%...,200,7165,<NA>,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1...
0,173.252.107.9,2022-03-31 00:00:13-03:00,GET /handle/10915/45833 HTTP/1.1,200,9539,<NA>,facebookexternalhit/1.1 (+http://www.facebook....
0,173.252.107.3,2022-03-31 00:00:13-03:00,GET /handle/10915/31292 HTTP/1.1,200,9512,<NA>,facebookexternalhit/1.1 (+http://www.facebook....
0,173.252.107.22,2022-03-31 00:00:14-03:00,GET /handle/10915/66507 HTTP/1.1,200,10170,<NA>,facebookexternalhit/1.1 (+http://www.facebook....
0,173.252.107.9,2022-03-31 00:00:14-03:00,GET /handle/10915/66507 HTTP/1.1,200,10170,<NA>,facebookexternalhit/1.1 (+http://www.facebook....
...,...,...,...,...,...,...,...
0,181.230.136.160,2022-03-30 20:20:22-03:00,GET /bitstream/handle/10915/49230/05%20Orienta...,200,1679,http://sedici.unlp.edu.ar/handle/10915/49230,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...
0,190.36.19.241,2022-03-30 20:20:22-03:00,GET /bitstream/handle/10915/1344/Conclusiones....,200,88849,https://www.google.com/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
0,144.76.14.40,2022-03-30 20:20:23-03:00,GET /discover?filter=http%3A%2F%2Fvoc.sedici.u...,200,8747,<NA>,Mozilla/5.0 (Windows NT 10.0; Win64; x64; tren...
0,52.70.240.171,2022-03-30 20:20:23-03:00,GET /handle/10915/61697/discover?filter=comuni...,200,7717,<NA>,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1...


Aplico regex de COUNTER para detectar bots conocidos por UserAgent

In [6]:
import re

def spambots_fail2ban(entry):
    # regex https://github.com/atmire/COUNTER-Robots/blob/master/generated/COUNTER_Robots_list.txt
    return bool(re.search("(bot|^Buck\/[0-9]|spider|crawl|^.?$|[^a]fish|^IDA$|^ruby$|^@ozilla\/\d|^脝脝陆芒潞贸碌脛$|^破解后的$|AddThis|A6-Indexer|ADmantX|alexa|Alexandria(\s|\+)prototype(\s|\+)project|AllenTrack|almaden|appie|API[\+\s]scraper|Arachni|Arachmo|architext|ArchiveTeam|aria2\/\d|arks|^Array$|asterias|atomz|axios\/\d|BDFetch|Betsie|baidu|biglotron|BingPreview|binlar|bjaaland|Blackboard[\+\s]Safeassign|blaiz-bee|bloglines|blogpulse|boitho\.com-dc|bookmark-manager|Brutus\/AET|BUbiNG|bwh3_user_agent|CakePHP|celestial|cfnetwork|checklink|checkprivacy|China\sLocal\sBrowse\s2\.6|Citoid|cloakDetect|coccoc\/1\.0|Code\sSample\sWeb\sClient|ColdFusion|collection@infegy.com|com\.plumanalytics|combine|contentmatch|ContentSmartz|convera|core|Cortana|CoverScout|crusty\/\d|curl\/|cursor|custo|DataCha0s\/2\.0|daum(oa)?|^\%?default\%?$|DeuSu\/|Dispatch\/\d|Docoloc|docomo|Download\+Master|Drupal|DSurf|DTS Agent|EasyBib[\+\s]AutoCite[\+\s]|easydl|EBSCO\sEJS\sContent\sServer|EcoSearch|ELinks\/|EmailSiphon|EmailWolf|Embedly|EThOS\+\(British\+Library\)|facebookexternalhit\/|favorg|Faveeo\/\d|FDM(\s|\+)\d|Feedbin|feedburner|FeedFetcher|feedreader|ferret|Fetch(\s|\+)API(\s|\+)Request|findlinks|findthatfile|^FileDown$|^Filter$|^firefox$|^FOCA|Fulltext|Funnelback|Genieo|GetRight|geturl|GigablastOpenSource|G-i-g-a-b-o-t|GLMSLinkAnalysis|Goldfire(\s|\+)Server|google|Grammarly|GroupHigh\/\d|grub|gulliver|gvfs\/|harvest|heritrix|holmes|htdig|htmlparser|HeadlessChrome|HttpComponents\/1.1|HTTPFetcher|http.?client|httpget|httpx|httrack|ia_archiver|ichiro|iktomi|ilse|Indy Library|^integrity\/\d|internetseer|intute|iSiloX|iskanie|^java\/\d{1,2}.\d|jeeves|Jersey\/\d|jobo|Koha|kyluka|larbin|libcurl|libhttp|libwww|lilina|^LinkAnalyser|link.?check|LinkLint-checkonly|^LinkParser\/|^LinkSaver\/|linkscan|LinkTiger|linkwalker|lipperhey|livejournal\.com|LOCKSS|LongURL.API|ltx71|lwp|lycos[_+]|MaCoCu|mail\.ru|MarcEdit|mediapartners-google|megite|MetaURI[\+\s]API\/\d\.\d|Microsoft(\s|\+)URL(\s|\+)Control|Microsoft Office Existence Discovery|Microsoft Office Protocol Discovery|Microsoft-WebDAV-MiniRedir|mimas|mnogosearch|moget|motor|^Mozilla$|^Mozilla.4\.0$|^Mozilla\/4\.0\+\(compatible;\)$|^Mozilla\/4\.0\+\(compatible;\+ICS\)$|^Mozilla\/4\.5\+\[en]\+\(Win98;\+I\)$|^Mozilla.5\.0$|^Mozilla\/5.0\+\(compatible;\+MSIE\+6\.0;\+Windows\+NT\+5\.0\)$|^Mozilla\/5\.0\+like\+Gecko$|^Mozilla\/5.0(\s|\+)Gecko\/20100115(\s|\+)Firefox\/3.6$|^MSIE|MuscatFerre|myweb|nagios|^NetAnts\/\d|netcraft|netluchs|newspaper\/\d|ng\/2\.|^Ning\/\d|no_user_agent|nomad|nutch|^oaDOI$|ocelli|Offline(\s|\+)Navigator|OgScrper|okhttp|onetszukaj|^Opera\/4$|OurBrowser|panscient|parsijoo|^Pattern\/\d|Pcore-HTTP|pear\.php\.net|perman|PHP\/|pidcheck|pioneer|playmusic\.com|playstarmusic\.com|^Postgenomic(\s|\+)v2|powermarks|proximic|PycURL|python|Qwantify|rambler|ReactorNetty\/\d|Readpaper|redalert|Riddler|robozilla|rss|scan4mail|scientificcommons|scirus|scooter|Scrapy\/\d|ScoutJet|^scrutiny\/\d|SearchBloxIntra|shoutcast|Site24x7|SkypeUriPreview|slurp|sogou|speedy|sqlmap|SrceDAMP|Strider|summify|sunrise|Sysomos|T\-H\-U\-N\-D\-E\-R\-S\-T\-O\-N\-E|tailrank|Teleport(\s|\+)Pro|Teoma|The[\+\s]Knowledge[\+\s]AI|titan|^Traackr\.com$|Trello|Trove|Turnitin|twiceler|Typhoeus|ucsd|ultraseek|^undefined$|^unknown$|Unpaywall|URL2File|urlaliasbuilder|urllib|^user.?agent$|^User-Agent|validator|virus.detector|voila|^voltron$|voyager\/|w3af\.org|Wanadoo|Web(\s|\+)Downloader|WebCloner|webcollage|WebCopier|Webinator|weblayers|Webmetrics|webmirror|webmon|weborama-fetcher|webreaper|WebStripper|WebZIP|Wget|WhatsApp|wordpress|worm|www\.gnip\.com|WWW-Mechanize|xenu|y!j|yacy|yahoo|yandex|Yeti\/\d|Zabbix|ZoteroTranslationServer|zeus|zyborg|7siters)", entry, re.IGNORECASE))

df_log['spambot'] = df_log['User_agent'].apply(lambda x: spambots_fail2ban(str(x)))

In [9]:
df_log

,ip,time,request,status,size,Referer,User_agent,spambot
0,3.224.220.101,2022-03-31 00:00:13-03:00,GET /handle/10915/60216/browse?authority=http%...,200,7165,None,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1...,True
1,173.252.107.9,2022-03-31 00:00:13-03:00,GET /handle/10915/45833 HTTP/1.1,200,9539,None,facebookexternalhit/1.1 (+http://www.facebook....,True
2,173.252.107.3,2022-03-31 00:00:13-03:00,GET /handle/10915/31292 HTTP/1.1,200,9512,None,facebookexternalhit/1.1 (+http://www.facebook....,True
3,173.252.107.22,2022-03-31 00:00:14-03:00,GET /handle/10915/66507 HTTP/1.1,200,10170,None,facebookexternalhit/1.1 (+http://www.facebook....,True
4,173.252.107.9,2022-03-31 00:00:14-03:00,GET /handle/10915/66507 HTTP/1.1,200,10170,None,facebookexternalhit/1.1 (+http://www.facebook....,True
...,...,...,...,...,...,...,...,...
89049,5.161.86.47,2022-03-31 09:35:03-03:00,GET /bitstream/10915/19075/1/1767-CUESTIONARIO...,301,445,None,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,False
89050,163.10.48.138,2022-03-31 09:35:04-03:00,GET /handle/10915/72466 HTTP/1.1,200,8206,None,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,False
89051,5.161.86.47,2022-03-31 09:35:04-03:00,GET /bitstream/handle/10915/19075/1767-CUESTIO...,200,167837,None,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,False
89052,5.161.87.70,2022-03-31 09:35:04-03:00,GET /bitstream/10915/18392/1/Sistemas+de+Argum...,301,411,None,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,False
